# TABLE OF CONTENTS:
---
* [Notebook Summary](#Notebook-Summary)
* [Setup](#Setup)
    * [Connect to Workspace](#Connect-to-Workspace)
* [Data](#Data)
    * [Overview](#Overview)
    * [Download & Extract Data](#Download-&-Extract-Data)
    * [Upload Data](#Upload-Data)
    * [Explore Data](#Explore-Data)
    * [Create and Register AML Dataset](#Create-and-Register-AML-Dataset)
---

# Notebook Summary

This notebook will download the [Stanford Dogs Dataset](http://vision.stanford.edu/aditya86/ImageNetDogs/) from the Stanford Vision website to the local compute and then upload it to the Azure Machine Learning (AML) workspace default blob storage. It will also create an AML file dataset that can be used for easy data access during the ML lifecycle.

Run this notebook from the Jupyter kernel that has been created in `00_environment_setup`.

# Setup

Append parent directory to sys path to be able to import created modules from src directory.

In [1]:
import sys
sys.path.append(os.path.dirname(os.path.abspath("")))

Automatically reload modules when changes are made.

In [2]:
%load_ext autoreload
%autoreload 2

Import libraries and modules.

In [3]:
# Import libraries
import azureml.core
import torchvision
from azureml.core import Dataset, Workspace

# Import created modules
from src.utils import download_stanford_dogs_archives, extract_stanford_dogs_archives, load_data, show_image, show_batch_of_images

print(f"azureml.core version: {azureml.core.VERSION}")

azureml.core version: 1.20.0


### Connect to Workspace

In order to connect and communicate with the AML workspace, a workspace object needs to be instantiated using the AML Python SDK.

In [4]:
# Connect to the AML workspace using interactive authentication
ws = Workspace.from_config()

# Data

### Overview

The [Stanford Dogs Dataset](http://vision.stanford.edu/aditya86/ImageNetDogs/) is an image dataset that will be used to train a multiclass dog breed classification model. In total there are 120 different dog breeds/classes and 20,580 images. The dataset has been built using images and annotations from ImageNet for the task of fine-grained image categorization. The images are three-channel color images of variable pixels in size. While a file with a given train/test split can be downloaded from the website, the train dataset will be further split into a validation and train set (80:20). This will ultimately lead into a data distribution as follows:
- 9600 training images (47.65%)
- 2400 validation images (11.66%)
- 8580 test images (41.69%)

### Download & Extract Data

Download the data to the local compute.

A utility file with functions to download the dogs dataset archive files from the Stanford Vision website and to extract the archives into a format expected by the [torchvision.datasets.ImageFolder](https://pytorch.org/docs/stable/torchvision/datasets.html#imagefolder) has been created (`<PROJECT_ROOT>/src/utils/data_utils.py`).

In [5]:
# Download the dataset archive files
download_stanford_dogs_archives()

0it [00:00, ?it/s]

100%|█████████▉| 792608768/793579520 [02:11<00:00, 5944311.46it/s]
0it [00:00, ?it/s]


 63%|██████▎   | 303104/481280 [00:00<00:00, 511654.20it/s]
483328it [00:01, 434535.41it/s]                            


In [ ]:
# Extract the dataset archives and remove them after extraction
extract_stanford_dogs_archives()

Lists.tar archive has been extracted successfully.
File lists have been read successfully.
Extracting images.tar archive...


793583616it [02:30, 5944311.46it/s]                               
  1%|▏         | 266/20701 [00:27<35:16,  9.66it/s]


  3%|▎         | 556/20701 [00:55<34:58,  9.60it/s]


  4%|▍         | 845/20701 [01:22<29:10, 11.34it/s]


  5%|▌         | 1123/20701 [01:51<24:29, 13.32it/s]


...

 16%|█▌        | 3284/20701 [06:13<57:38,  5.04it/s]


 17%|█▋        | 3449/20701 [06:39<36:49,  7.81it/s]


 18%|█▊        | 3665/20701 [07:05<25:44, 11.03it/s]


 19%|█▉        | 3883/20701 [07:32<37:31,  7.47it/s]


 19%|█▉        | 3976/20701 [07:45<43:55,  6.35it/s]

### Upload Data

Upload the data to the default AML datastore.

In [ ]:
datastore = ws.get_default_datastore()
datastore.upload(src_dir="../data", target_path="data/stanford_dogs", overwrite=True)

### Explore Data

Load the data into memory. A utility function to create dataloaders has been created as part of the `<PROJECT_ROOT>/src/utils/data_utils.py` script.

In [ ]:
# Load data
dataloaders, dataset_sizes, class_names = load_data("../data")

Display an example image. All images have different shapes.

In [ ]:
show_image(image_path="../data/val/n02085620-Chihuahua/n02085620_1152.jpg")

Display the first batch of 4 images.

In [ ]:
# Get some random training images
dataiter = iter(dataloaders["val"])
images, labels = dataiter.next()

# Show images
show_batch_of_images(torchvision.utils.make_grid(images))
# Print labels
print("\n".join("%s" % class_names[labels[j]].split("-")[1] for j in range(4)))

### Create and Register AML Dataset

Register the data as a file dataset in the AML workspace for easy accessibility throughout the ML lifecycle.

In [ ]:
# Create a dataset object from the datastore location
dataset = Dataset.File.from_files(path=(datastore, "data/stanford_dogs"))

In [ ]:
# Register the dataset
dataset = dataset.register(workspace=ws,
                           name="stanford-dogs-dataset",
                           description="Stanford Dogs Dataset containing training, validation and test data",
                           create_new_version=True)